## Home Credit

**将问题框架化并关注重点**

1. 用业务术语定义目标
    * 预测给定信息用户的还款能力，以概率的方式
2. 你的解决方案将如何使用？
    * 用来预测未来贷款申请客户的好坏，作为申请决策辅助依据。
3. 目前的解决方案/解决方法（如果有的话）是什么？
    * 未知
4. 你应该如何解决这个问题（监督/非监督，在线/离线等）？
    * 使用监督学习，回归问题，label中‘0’表示好人，‘1’表示坏人，给出的数介于两者之间，清洗好特征后，多模型测试
5. 如何度量模型的表现？
    * 使用roc下的面积
6. 模型的表现是否和业务目标一致？
    * 纯模型训练，无实际业务目标，暂且认为一致
7. 达到业务目标所需的最低性能是多少？
    * 未知，一般任务auc=0.75
8. 类似的问题如何解决？是否可以复用经验或工具？
    * pass
9. 人员是否专业？
    * pass
10. 你如何动手解决问题？
    * pass
11. 列出目前你（或者其他人）所做的假设
    * pass
12. 如果可能，验证假设。

---

观察到的点：

1. 这个比赛最难的地方是对数据的理解，给出的7张表字段很多，但是解释却很简单，很多字段的值也意义不明，很难在此基础上加工新特征，需要一定的时间和金融业务知识理解数据
2. 这个比赛对于新手比较友好，有很多比较好的notebook可以参考，例如[Gentle Introduction](https://www.kaggle.com/code/willkoehrsen/start-here-a-gentle-introduction/notebook) [LightGBM with Simple Features](https://www.kaggle.com/code/jsaguiar/lightgbm-with-simple-features?scriptVersionId=6025993)
3. 总体来看，给出的数据可以分4个部分，时间 金额 笔数 其他
4. 猜测有一些预测力比较高的变量，例如EXT_SOURCE_2，可能是购买的外部数据，例如FICO分之类的东西，FLAG_DOCUMENT_3变量可能是收入证明之类的东西
5. Bureau类似解析征信报告得到的数据，例如一个人申请了多笔贷款，有房贷 车贷 抵押贷等，那么它在这个表中就有多条数据，Bureau_balance类似征信报告中每条贷款的附加标记数据，如果一个人在Bureau表中没有数据，要么这个人从未贷过款，要么借贷的渠道没有上报数据
6. 工程量最大的是针对每个表加工变量，需要将多条数据聚合成有意义的 申请人粒度的单条数据，中间需要做大量数据探索的工作，如果可能可以将数据导入数据库中加工计算
7. 发现有一些时间样本的值为值为365243.0，明显异常，根据[官方给出的解释](https://www.kaggle.com/c/home-credit-default-risk/discussion/57248)


In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import warnings
warnings.filterwarnings('ignore')


In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler,OneHotEncoder,FunctionTransformer
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
import sklearn
sklearn.set_config(display="diagram")

In [3]:
from sklearn.linear_model import LinearRegression,SGDRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPRegressor

In [4]:
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
import time


def cross_validate_with_feature_importance(models, data, labels, n_folds=5):
    skf = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    for model_idx, model in enumerate(models):
        print(f"Model {model_idx + 1}: {type(model).__name__}")
        total_feature_importance = None
        scores = []
        
        for fold, (train_idx, val_idx) in enumerate(skf.split(data, labels)):
            print(f"Fold {fold + 1}")
            X_train, y_train = data.iloc[train_idx], labels.iloc[train_idx].values
            X_val, y_val = data.iloc[val_idx], labels.iloc[val_idx].values

            start_time = time.time()
            model.fit(X_train, y_train)
            train_time = time.time() - start_time
            print(f"Training time: {train_time:.2f}s")

            if fold == 0 and train_time > 3600:
                print(f"Model training skipped because it took {train_time:.2f}s")
                break

            if hasattr(model, "predict_proba"):
                y_pred = model.predict_proba(X_val)[:, 1]
            else:
                y_pred = model.predict(X_val)
            score = roc_auc_score(y_val, y_pred)
            scores.append(score)
            print(f"Validation ROC AUC score: {score:.4f}")

            if hasattr(model, "feature_importances_"):
                fold_feature_importance = pd.Series(model.feature_importances_, index=data.columns)
                if total_feature_importance is None:
                    total_feature_importance = fold_feature_importance
                else:
                    total_feature_importance += fold_feature_importance

        if total_feature_importance is not None:
            top_features = total_feature_importance.sort_values(ascending=False)[:15]
            print("Top 15 most important features:")
            print(top_features)
        if scores:
            print(f"Mean ROC AUC score: {np.mean(scores):.4f} ± {np.std(scores):.4f}")
        else:
            print("No validation scores obtained.")


In [5]:
def reduce_mem_usage(df, ignore_cols=['SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV']):
    """在不损失数据信息的情况下，通过转换数字数据类型来减少数据帧的内存使用"""
    # 初始化数据框的内存使用
    start_mem = df.memory_usage().sum() / 1024**2
    print('初始内存使用: {:.2f} MB'.format(start_mem))
    # 剔除特定列
    cols = [ col for col in df.columns if col not in ignore_cols]
    # 遍历每一列
    for col in cols:
        col_type = df[col].dtype
        
        # 如果数据类型是整数类型
        if col_type != object and col_type.name != 'category' and 'datetime' not in col_type.name:
            c_min = df[col].min()
            c_max = df[col].max()
            
            # 如果最小值和最大值都可以用更小的数据类型表示
            if str(col_type)[:3] == 'int' and c_min >= np.iinfo(np.int8).min and c_max <= np.iinfo(np.int8).max:
                df[col] = df[col].astype(np.int8)
            elif str(col_type)[:3] == 'int' and c_min >= np.iinfo(np.int16).min and c_max <= np.iinfo(np.int16).max:
                df[col] = df[col].astype(np.int16)
            elif str(col_type)[:3] == 'int' and c_min >= np.iinfo(np.int32).min and c_max <= np.iinfo(np.int32).max:
                df[col] = df[col].astype(np.int32)
            elif str(col_type)[:3] == 'int' and c_min >= np.iinfo(np.int64).min and c_max <= np.iinfo(np.int64).max:
                df[col] = df[col].astype(np.int64)
            
            # 如果最小值和最大值都可以用更小的数据类型表示
            elif str(col_type)[:5] == 'float' and c_min >= np.finfo(np.float16).min and c_max <= np.finfo(np.float16).max:
                df[col] = df[col].astype(np.float16)
            elif str(col_type)[:5] == 'float' and c_min >= np.finfo(np.float32).min and c_max <= np.finfo(np.float32).max:
                df[col] = df[col].astype(np.float32)
            else:
                pass

    # 输出优化后的内存使用
    end_mem = df.memory_usage().sum() / 1024**2
    print('优化后的内存使用: {:.2f} MB'.format(end_mem))
    return df

In [6]:
def select_low_cardinality_numeric_features(df, label_col, threshold=5):
    """
    挑选出除了 label_col 以外的数值型特征，并判断其唯一值数是否低于 threshold，
    如果是，则将该特征的名称加入到列表 low_cardinality_feats 中返回。

    Parameters:
    ----------
    df: pandas.DataFrame
        数据表，包含了特征和目标变量
    label_col: str
        目标变量的名称
    threshold: int
        指定唯一值数量的阈值，低于该值的特征将被视为“唯一值较少的特征”，默认值为 5。

    Returns:
    ----------
    low_cardinality_feats: list
        唯一值较少的数值型特征的名称列表
    """
    numeric_feats = df.select_dtypes(include='number').columns.tolist()
    low_cardinality_feats = []
    for feat in numeric_feats:
        if feat == label_col:
            continue
        if df[feat].nunique() <= threshold:
            low_cardinality_feats.append(feat)
    return low_cardinality_feats


**获取数据**

注意：尽可能自动化，以便你轻松获取新数据。

1. 列出你需要的数据和数据量。
    * 已给定
2. 查找并记录你可以获取该数据的位置。
    * pass
3. 检查它将占用多少存储空间。
    * 2.5G
4. 检查法律义务并在必要时获取授权。
    *  开放数据集，无需授权
5. 获取访问权限。
    * kaggle
6. 创建工作目录（拥有足够的存储空间）。
    * pass
7. 获取数据。
    * 如下
8. 将数据转换为你可以轻松操作的格式（不更改数据本身）。
    * pass
9. 确保删除或保护敏感信息（比如，匿名）。
    * 数据集本身已处理
10. 检查数据的大小和类型（时间序列，样本，地理信息等）。
    * 如下
11. 抽样出测试集，将它放在一边，以后不需要关注它（没有数据窥探！）。
    * 如下

---

In [7]:
# 使用kaggle命令获取数据并解压
# &&kaggle competitions download -c home-credit-default-risk\
# &&mkdir data&&unzip *.zip -d ./data/

In [8]:
# 查看数据占用空间 
!du -sh ./data/

2.5G	./data/


In [9]:
def load_data(data_set_name:str):
    return pd.read_csv('./data/{}.csv'.format(data_set_name))

In [10]:
application = load_data('application_train')
# application_test = load_data('application_test')
application.shape
# application.columns.difference(application_test.columns)
# application_test.shape
# del application_test

(307511, 122)

In [11]:
application = reduce_mem_usage(application)

初始内存使用: 286.23 MB
优化后的内存使用: 93.55 MB


**快速看一眼数据**

In [12]:
application.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 307511 entries, 0 to 307510
Columns: 122 entries, SK_ID_CURR to AMT_REQ_CREDIT_BUREAU_YEAR
dtypes: float16(61), float32(4), int16(2), int32(1), int64(1), int8(37), object(16)
memory usage: 93.6+ MB


In [13]:
def trans_inf_days(df):
    # 雇佣时长是否有极大值
    df['isinf_DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].where(df['DAYS_EMPLOYED'] == 365243.0,0).where(df['DAYS_EMPLOYED'] != 365243.0,1)
    # 将雇佣时长极大值替换成空
    df['DAYS_EMPLOYED'] = df['DAYS_EMPLOYED'].where(df['DAYS_EMPLOYED'] != 365243.0, np.nan)
    return df
application = trans_inf_days(application)

**创建测试集**

因为kaggle比赛的application_test没有target，无法测量ROC下的面积，所以丢弃不用，切分application_train为训练集和测试集

在切分数据集的时候有多种策略可以选择，一般而言，可以随机切分 也可以分层切分，在本场景下，数据集比较大，坏样本相对较多，所以随机切；

**随机切分**

In [14]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(application, test_size=0.2, random_state= 42)

del application

**探索数据**

注意：尝试从领域专家那获取有关这些步骤的见解。

1. 创建用于探索的数据副本（如有必要，将其取样为可管理的大小）。
2. 创建一个 Jupyter 笔记本来记录你的数据探索。
3. 研究每个属性及其特征：
  * 名称；
  * 类型（分类，整数/浮点数，有界/无界，文本，结构化数据等）；
  * 缺失数据的百分比；
  * 噪声点和它的类型（随机点，异常点，舍入误差等）；
  * 对任务可能有用吗？
  * 分布类型（高斯分布，均匀分布，对数分布等）。
4. 对于监督学习任务，确定目标属性。
5. 可视化数据。
6. 研究属性间的相关性。
7. 研究怎如何手动解决问题。
8. 确定你想要应用的有效的转换。
9. 确定有用的额外数据（回到上一步）。
10. 记录你所学到的知识。


## EDA

In [15]:
app = train_set.copy()

In [16]:
app.dtypes.value_counts()

float16    61
int8       37
object     16
float32     4
int16       2
int64       1
float64     1
int32       1
dtype: int64

In [17]:
def missing_values_summary(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100*df.isnull().sum() / len(df)
    mis_val_table =pd.concat([mis_val, mis_val_percent], axis=1)   
    mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
    mis_val_table_ren_columns = mis_val_table_ren_columns[mis_val_table_ren_columns.iloc[:,1] !=0].sort_values(
    '% of Total Values', ascending = False).round(1)
    mis_val_table_ren_columns = mis_val_table_ren_columns.merge(df.dtypes.rename('dtypes').to_frame(),left_index=True, right_index=True)
    print('Your selected dataframe has ' + str(df.shape[1])+ ' columns.\n'
         "There are " + str(mis_val_table_ren_columns.shape[0])+ ' columns that have missing values.')
    return mis_val_table_ren_columns

In [18]:
missing_values_summary(app)

Your selected dataframe has 123 columns.
There are 68 columns that have missing values.


,Missing Values,% of Total Values,dtypes
COMMONAREA_MODE,171929,69.9,float16
COMMONAREA_AVG,171929,69.9,float16
COMMONAREA_MEDI,171929,69.9,float16
NONLIVINGAPARTMENTS_MEDI,170868,69.5,float16
NONLIVINGAPARTMENTS_MODE,170868,69.5,float16
NONLIVINGAPARTMENTS_AVG,170868,69.5,float16
FONDKAPREMONT_MODE,168286,68.4,object
LIVINGAPARTMENTS_MEDI,168196,68.4,float16
LIVINGAPARTMENTS_AVG,168196,68.4,float16
LIVINGAPARTMENTS_MODE,168196,68.4,float16


-----

**简单筛选一下特征**

In [19]:
# 筛选特征
cate_cols = []
num_cols = []

# 将类别属性放进去
cate_cols.extend(app.dtypes[app.dtypes == 'object'].index.tolist())

# 将唯一值少于5个的数值型变量也放进去
cate_cols.extend(select_low_cardinality_numeric_features(app, 'TARGET'))

num_cols.extend(app.columns.difference(cate_cols))
num_cols.remove('TARGET')

cat_cols_object = app[cate_cols].select_dtypes(include=['object']).columns
cat_cols_number = app[cate_cols].select_dtypes(include=['number']).columns
del app

---

**准备数据**

注意：
* 处理数据副本（保持原始数据集完整）。
* 为你应用的所有数据转换编写函数，原因有五：
    * 你可以在下次获得新数据集时轻松准备数据
    * 你可以在未来的项目中应用这些转换
    * 用来清洗和准备测试数据集
    * 一旦项目上线你可以用来清洗和准备新的数据集
    * 为了便于将你的准备选择视为超参数

1. 数据清洗：
    * 修正或移除异常值（可选）。
    * 填补缺失值（比如用零，平均值，中位数等）或者删除所在行（或者列）。
2. 特征提取（可选）：
    * 丢弃不提供有用信息的属性；
3. 适当的特征工程：
    * 连续特征离散化。
    * 分解特征（比如分类，日期/时间等）。
    * 对特征添加有益的转换（比如 log(x)，sqrt(x)，x^2 等）
    * 将一些特征融合为有益的新特征
4. 特征缩放：
    * 标准化或者正规化特征。

**准备数据副本**

In [25]:
app = train_set.drop('TARGET', axis=1)
app_labels = train_set['TARGET'].copy()

In [26]:
app.shape

(246008, 122)

**数据清洗**

1. 上面已经列清楚了哪些列有缺失值，分析具体情况进行填充
2. 针对数值型的列，使用具体情况具体分析进行填充，一般来说，数据服从正态分布，使用均值；不服从使用中位数，如果是离散的数据点，可以使用众数；如果缺失较少，使用均值或者中位数，可以保证数据整体分布，缺失较多，可以使用众数，分布更加稳定
3. 针对对象型，使用特殊标记‘UKN’进行填充

## Pipline

In [27]:
num_pipeline = make_pipeline(
                SimpleImputer(strategy='median'),
                StandardScaler()
                )
cat_number_pipeline = make_pipeline(
                FunctionTransformer(lambda X: X.astype(str),feature_names_out='one-to-one'),
                SimpleImputer(strategy='constant', fill_value='UKN'),
                OneHotEncoder(handle_unknown='ignore')
)
cat_object_pipeline = make_pipeline(
                SimpleImputer(strategy='constant', fill_value='UKN'),
                OneHotEncoder(handle_unknown='ignore')
)
processing = ColumnTransformer([
    ('num', num_pipeline, num_cols),
    ('cate_object',cat_object_pipeline,cat_cols_object),
    ('cate_number', cat_number_pipeline, cat_cols_number)
], remainder='passthrough')

In [28]:
app_transform = processing.fit_transform(app)
app_transform.shape
app_transformed = pd.DataFrame(app_transform, columns=processing.get_feature_names_out())

(246008, 293)

**列出有用模型**

注意：

  * 如果数据量巨大，你可能需要采样出较小的训练集，以便在合理的时间内训练许多不同的模型（请注意，这会对诸如大型神经网络或随机森林等复杂模型进行处罚）。
  * 再次尝试尽可能自动化这些步骤。

1. 使用标准参数训练许多快速、粗糙的模型（比如线性模型，朴素贝叶斯模型，支持向量机模型，随机森林模型，神经网络等）。
2. 衡量并比较他们的表现。
  * 对于每个模型，使用 N 折交叉验证法，并且计算基于 N 折交叉验证的均值与方差。
3. 分析每种算法的最重要变量。
4. 分析模型产生的错误类型。
  * 人们用什么数据来避免这些错误？
5. 进行一轮快速的特征提取和特征工程。
6. 对之前的五个步骤进行一两次的快速迭代。
7. 列出前三到五名最有用的模型，由其是产生不同类型错误的模型。


## Baseline

In [29]:
lr = LinearRegression()
sgd = SGDRegressor()
tree = DecisionTreeRegressor()
# svr = SVR()
# gpr = GaussianProcessRegressor()
gnb = GaussianNB()
nn = MLPRegressor()
models = [lr, sgd, tree, gnb, nn]

In [30]:
# cross_validate_with_feature_importance(models, app_transformed, app_labels)

In [31]:
del app,app_transform,app_transformed,app_labels

先粗略的跑一下模型，不做任何处理，LR：0.7461 ± 0.0038  SGD:0.7349 ± 0.0054  tree:0.5394 ± 0.0020 gnb:0.5290 ± 0.0042  nn:0.7108 ± 0.0133

## add more information

在生成新特征的时候，变量命名遵循以下规则，方便我们后面快速筛选变量
1. 时间相关变量名必须包含'DAYS'
2. 金额相关变量名必须包含'AMT'
3. 次数相关变量名必须包含'NUM'

In [15]:
import gc

def concat_df_by_name(name_str,all_vars):
    df_list = []
    keys_to_delete = []  # 存储需要删除的键值对
    for var_name, var_value in all_vars.items():
        if isinstance(var_value, pd.DataFrame) and name_str in var_name:
            df_list.append(var_value)
            keys_to_delete.append(var_name)  # 记录需要删除的键
    for key in keys_to_delete:  # 在循环结束后删除键值对
        del all_vars[key]
    if not df_list:
        return pd.DataFrame()
    result = pd.concat(df_list, axis=1)
    gc.collect()
    return result

In [16]:
def process_bureau():
    # bureau表
    bureau = load_data('bureau')
    bureau = reduce_mem_usage(bureau)

    # 剔除17条更新时间明显异常的数据
    bureau = bureau[bureau.DAYS_CREDIT_UPDATE <= 0]

    # 征信返回总笔数
    BUREAU_NUM = bureau[['SK_ID_CURR','SK_ID_BUREAU']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_NUM'})
    # 征信报告中活跃贷款笔数
    BUREAU_ACTIVE_NUM = bureau[bureau['CREDIT_ACTIVE']=='Active'][['SK_ID_CURR','SK_ID_BUREAU']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_ACTIVE_NUM'})
    # 征信报告中关闭贷款笔数
    BUREAU_COLSED_NUM = bureau[bureau['CREDIT_ACTIVE']=='Closed'][['SK_ID_CURR','SK_ID_BUREAU']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_Closed_NUM'})
    # 征信报告中出售贷款笔数
    BUREAU_SOLD_NUM = bureau[bureau['CREDIT_ACTIVE']=='Sold'][['SK_ID_CURR','SK_ID_BUREAU']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_SOLD_NUM'})
    # 征信报告中坏账贷款笔数
    BUREAU_BAD_DEBT_NUM = bureau[bureau['CREDIT_ACTIVE']=='Bad debt'][['SK_ID_CURR','SK_ID_BUREAU']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_BAD_DEBT_NUM'})
    # 最早一次贷款距今时长
    BUREAU_MINDAYS_APPLICATION = bureau[['SK_ID_CURR','DAYS_CREDIT']].groupby(['SK_ID_CURR']).min().rename(columns={'DAYS_CREDIT':'BUREAU_MINDAYS_APPLICATION'})
    # 最近一次贷款距今时长
    BUREAU_MAXDAYS_APPLICATION = bureau[['SK_ID_CURR','DAYS_CREDIT']].groupby(['SK_ID_CURR']).max().rename(columns={'DAYS_CREDIT':'BUREAU_MAXDAYS_APPLICATION'})
    # 历史上有过逾期的贷款笔数
    BUREAU_NUM_OVERDUE = bureau[bureau['CREDIT_DAY_OVERDUE'] > 0][['SK_ID_CURR','CREDIT_DAY_OVERDUE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_DAY_OVERDUE':'BUREAU_NUM_OVERDUE'})
    # 历史上最长逾期天数
    BUREAU_MAXDAY_OVERDUE = bureau[bureau['CREDIT_DAY_OVERDUE'] > 0][['SK_ID_CURR','CREDIT_DAY_OVERDUE']].groupby(['SK_ID_CURR']).max().rename(columns={'CREDIT_DAY_OVERDUE':'BUREAU_MAXDAY_OVERDUE'})
    # 提前还款的笔数
    BUREAU_NUM_PREPAY = bureau[bureau['DAYS_CREDIT_ENDDATE'] > bureau['DAYS_ENDDATE_FACT'] ][['SK_ID_CURR','DAYS_ENDDATE_FACT']].groupby(['SK_ID_CURR']).count().rename(columns={'DAYS_ENDDATE_FACT':'BUREAU_NUM_PREPAY'})
    # 到期还款笔数
    BUREAU_NUM_NORMAL = bureau[bureau['DAYS_CREDIT_ENDDATE'] == bureau['DAYS_ENDDATE_FACT'] ][['SK_ID_CURR','DAYS_ENDDATE_FACT']].groupby(['SK_ID_CURR']).count().rename(columns={'DAYS_ENDDATE_FACT':'BUREAU_NUM_NORMAL'})
    # 延后还款笔数
    BUREAU_NUM_DELAY = bureau[bureau['DAYS_CREDIT_ENDDATE'] < bureau['DAYS_ENDDATE_FACT'] ][['SK_ID_CURR','DAYS_ENDDATE_FACT']].groupby(['SK_ID_CURR']).count().rename(columns={'DAYS_ENDDATE_FACT':'BUREAU_NUM_DELAY'})
    # 历史最大逾期金额
    BUREAU_MAXAMT_OVERDUE = bureau[bureau['AMT_CREDIT_MAX_OVERDUE'] > 0][['SK_ID_CURR','AMT_CREDIT_MAX_OVERDUE']].groupby(['SK_ID_CURR']).max().rename(columns={'AMT_CREDIT_MAX_OVERDUE':'BUREAU_MAXAMT_OVERDUE'})
    # 总展期次数
    BUREAU_PROLONG_NUM = bureau[['SK_ID_CURR','CNT_CREDIT_PROLONG']].groupby(['SK_ID_CURR']).max().rename(columns={'CNT_CREDIT_PROLONG':'BUREAU_PROLONG_NUM'})
    # 总还款金额
    BUREAU_LOAN_AMT = bureau[['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_LOAN_AMT'})
    # 总未还金额
    BUREAU_DEBT_AMT = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBT_AMT'})
    # 最大信用额度
    BUREAU_LIMITAMT_MAX = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_LIMIT']].groupby(['SK_ID_CURR']).max().rename(columns={'AMT_CREDIT_SUM_LIMIT':'BUREAU_LIMITAMT_MAX'})
    # 总逾期金额
    BUREAU_SUMAMT_OVERDUE = bureau[['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_SUMAMT_OVERDUE'})
    # 消费贷数
    BUREAU_NUM_CONSUMER = bureau[bureau['CREDIT_TYPE'] == 'Consumer credit' ][['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_TYPE':'BUREAU_NUM_CONSUMER'})
    # 信用卡数
    BUREAU_NUM_CARD = bureau[bureau['CREDIT_TYPE'] == 'Credit card' ][['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_TYPE':'BUREAU_NUM_CARD'})
    # 汽车贷款数
    BUREAU_NUM_CAR = bureau[bureau['CREDIT_TYPE'] == 'Car loan' ][['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_TYPE':'BUREAU_NUM_CAR'})
    # 抵押贷款数
    BUREAU_NUM_MORTGAGE = bureau[bureau['CREDIT_TYPE'] == 'Mortgage' ][['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_TYPE':'BUREAU_NUM_MORTGAGE'})
    # 小微贷款数
    BUREAU_NUM_MiCROLOAN = bureau[bureau['CREDIT_TYPE'] == 'Microloan' ][['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_TYPE':'BUREAU_NUM_MiCROLOAN'})
    # 其他贷款数
    BUREAU_NUM_OTHER = bureau[~bureau.CREDIT_TYPE.isin(['Consumer credit','Credit card','Car loan','Mortgage','Microloan']) ][['SK_ID_CURR','CREDIT_TYPE']].groupby(['SK_ID_CURR']).count().rename(columns={'CREDIT_TYPE':'BUREAU_NUM_OTHER'})
    # 消费贷借款总金额
    BUREAU_AMT_CONSUMER = bureau[bureau['CREDIT_TYPE'] == 'Consumer credit' ][['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_AMT_CONSUMER'})
    # 信用卡借款总金额
    BUREAU_AMT_CARD = bureau[bureau['CREDIT_TYPE'] == 'Credit card' ][['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_AMT_CARD'})
    # 汽车贷款借款总金额
    BUREAU_AMT_CAR = bureau[bureau['CREDIT_TYPE'] == 'Car loan' ][['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_AMT_CAR'})
    # 抵押贷款借款总金额
    BUREAU_AMT_MORTGAGE = bureau[bureau['CREDIT_TYPE'] == 'Mortgage' ][['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_AMT_MORTGAGE'})
    # 小微贷款借款总金额
    BUREAU_AMT_MiCROLOAN = bureau[bureau['CREDIT_TYPE'] == 'Microloan' ][['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_AMT_MiCROLOAN'})
    # 其他贷款借款总金额
    BUREAU_AMT_OTHER = bureau[~bureau.CREDIT_TYPE.isin(['Consumer credit','Credit card','Car loan','Mortgage','Microloan']) ][['SK_ID_CURR','AMT_CREDIT_SUM']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM':'BUREAU_AMT_OTHER'})
    # 消费贷借款未还总金额
    BUREAU_DEBTAMT_CONSUMER = bureau[bureau['CREDIT_TYPE'] == 'Consumer credit' ][['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBTAMT_CONSUMER'})
    # 信用卡借款未还总金额
    BUREAU_DEBTAMT_CARD = bureau[bureau['CREDIT_TYPE'] == 'Credit card' ][['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBTAMT_CARD'})
    # 汽车贷款借款未还总金额
    BUREAU_DEBTAMT_CAR = bureau[bureau['CREDIT_TYPE'] == 'Car loan' ][['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBTAMT_CAR'})
    # 抵押贷款借款未还总金额
    BUREAU_DEBTAMT_MORTGAGE = bureau[bureau['CREDIT_TYPE'] == 'Mortgage' ][['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBTAMT_MORTGAGE'})
    # 小微贷款借款未还总金额
    BUREAU_DEBTAMT_MiCROLOAN = bureau[bureau['CREDIT_TYPE'] == 'Microloan' ][['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBTAMT_MiCROLOAN'})
    # 其他贷款借款未还总金额
    BUREAU_DEBTAMT_OTHER = bureau[~bureau.CREDIT_TYPE.isin(['Consumer credit','Credit card','Car loan','Mortgage','Microloan']) ][['SK_ID_CURR','AMT_CREDIT_SUM_DEBT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_DEBT':'BUREAU_DEBTAMT_OTHER'})
    # 消费贷逾期总金额
    BUREAU_OVERDUEAMT_CONSUMER = bureau[bureau['CREDIT_TYPE'] == 'Consumer credit' ][['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_OVERDUEAMT_CONSUMER'})
    # 信用卡逾期总金额
    BUREAU_OVERDUEAMT_CARD = bureau[bureau['CREDIT_TYPE'] == 'Credit card' ][['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_OVERDUEAMT_CARD'})
    # 汽车贷款逾期总金额
    BUREAU_OVERDUEAMT_CAR = bureau[bureau['CREDIT_TYPE'] == 'Car loan' ][['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_OVERDUEAMT_CAR'})
    # 抵押贷款逾期总金额
    BUREAU_OVERDUEAMT_MORTGAGE = bureau[bureau['CREDIT_TYPE'] == 'Mortgage' ][['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_OVERDUEAMT_MORTGAGE'})
    # 小微贷款逾期总金额
    BUREAU_OVERDUEAMT_MiCROLOAN = bureau[bureau['CREDIT_TYPE'] == 'Microloan' ][['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_OVERDUEAMT_MiCROLOAN'})
    # 其他贷款逾期总金额
    BUREAU_OVERDUEAMT_OTHER = bureau[~bureau.CREDIT_TYPE.isin(['Consumer credit','Credit card','Car loan','Mortgage','Microloan']) ][['SK_ID_CURR','AMT_CREDIT_SUM_OVERDUE']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT_SUM_OVERDUE':'BUREAU_OVERDUEAMT_OTHER'})
    # 最近一次更新记录时间
    BUREAU_LAST_UPDATE_DAYS = bureau[['SK_ID_CURR','DAYS_CREDIT_UPDATE']].groupby(['SK_ID_CURR']).max().rename(columns={'DAYS_CREDIT_UPDATE':'BUREAU_LAST_UPDATE_DAYS'})

    del bureau
    all_vars = locals()
    bureau_extra = concat_df_by_name('BUREAU',all_vars)
    print(bureau_extra.shape)
    return bureau_extra

In [17]:
def process_bureau_balance():
    # bureau_balance表
    bureau_balance = load_data('bureau_balance')
    bureau_balance = reduce_mem_usage(bureau_balance)
    bureau = load_data('bureau')
    bureau = reduce_mem_usage(bureau)

    tmp = bureau[['SK_ID_BUREAU','SK_ID_CURR']]
    bureau_balance_union = pd.merge(bureau_balance, tmp, how='left', on='SK_ID_BUREAU')
    bureau_balance_union['SK_ID_CURR'] = bureau_balance_union['SK_ID_CURR'].fillna(0).astype('int64')
    bureau_balance_union = bureau_balance_union[bureau_balance_union.STATUS.isin(['0','1','2','3','4','5'])]

    # 征信最近1个月逾期状态 0 1 2 3 4 5的笔数
    tmp = bureau_balance_union[bureau_balance_union.MONTHS_BALANCE >= -1][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
    BUREAU_OVERDUE_NUM_1_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_1_0'})
    BUREAU_OVERDUE_NUM_1_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_1_1'})
    BUREAU_OVERDUE_NUM_1_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_1_2'})
    BUREAU_OVERDUE_NUM_1_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_1_3'})
    BUREAU_OVERDUE_NUM_1_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_1_4'})
    BUREAU_OVERDUE_NUM_1_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_1_5'})

    # 征信最近1-3个月逾期状态 0 1 2 3 4 5的笔数
    tmp = bureau_balance_union[ (bureau_balance_union.MONTHS_BALANCE < -1) & (bureau_balance_union.MONTHS_BALANCE >= -3) ][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
    BUREAU_OVERDUE_NUM_3_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_3_0'})
    BUREAU_OVERDUE_NUM_3_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_3_1'})
    BUREAU_OVERDUE_NUM_3_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_3_2'})
    BUREAU_OVERDUE_NUM_3_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_3_3'})
    BUREAU_OVERDUE_NUM_3_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_3_4'})
    BUREAU_OVERDUE_NUM_3_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_3_5'})

    # 征信最近3-6个月逾期状态 0 1 2 3 4 5的笔数
    tmp = bureau_balance_union[ (bureau_balance_union.MONTHS_BALANCE < -3) & (bureau_balance_union.MONTHS_BALANCE >= -6) ][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
    BUREAU_OVERDUE_NUM_6_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_6_0'})
    BUREAU_OVERDUE_NUM_6_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_6_1'})
    BUREAU_OVERDUE_NUM_6_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_6_2'})
    BUREAU_OVERDUE_NUM_6_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_6_3'})
    BUREAU_OVERDUE_NUM_6_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_6_4'})
    BUREAU_OVERDUE_NUM_6_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_6_5'})

    # 征信最近6-12个月逾期状态 0 1 2 3 4 5的笔数
    tmp = bureau_balance_union[ (bureau_balance_union.MONTHS_BALANCE < -6) & (bureau_balance_union.MONTHS_BALANCE >= -12) ][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
    BUREAU_OVERDUE_NUM_12_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_12_0'})
    BUREAU_OVERDUE_NUM_12_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_12_1'})
    BUREAU_OVERDUE_NUM_12_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_12_2'})
    BUREAU_OVERDUE_NUM_12_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_12_3'})
    BUREAU_OVERDUE_NUM_12_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_12_4'})
    BUREAU_OVERDUE_NUM_12_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_12_5'})

    # 征信最近12-24个月逾期状态 0 1 2 3 4 5的笔数
    tmp = bureau_balance_union[ (bureau_balance_union.MONTHS_BALANCE < -12) & (bureau_balance_union.MONTHS_BALANCE >= -24) ][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
    BUREAU_OVERDUE_NUM_24_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_24_0'})
    BUREAU_OVERDUE_NUM_24_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_24_1'})
    BUREAU_OVERDUE_NUM_24_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_24_2'})
    BUREAU_OVERDUE_NUM_24_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_24_3'})
    BUREAU_OVERDUE_NUM_24_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_24_4'})
    BUREAU_OVERDUE_NUM_24_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_24_5'})

    # 征信最近24-36个月逾期状态 0 1 2 3 4 5的笔数
    tmp = bureau_balance_union[ (bureau_balance_union.MONTHS_BALANCE < -24) & (bureau_balance_union.MONTHS_BALANCE >= -36) ][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
    BUREAU_OVERDUE_NUM_36_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_36_0'})
    BUREAU_OVERDUE_NUM_36_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_36_1'})
    BUREAU_OVERDUE_NUM_36_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_36_2'})
    BUREAU_OVERDUE_NUM_36_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_36_3'})
    BUREAU_OVERDUE_NUM_36_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_36_4'})
    BUREAU_OVERDUE_NUM_36_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_36_5'})

    # 征信最近36+个月逾期状态 0 1 2 3 4 5的笔数
    tmp = bureau_balance_union[ bureau_balance_union.MONTHS_BALANCE < -36][['SK_ID_BUREAU','SK_ID_CURR','STATUS']].groupby(['SK_ID_BUREAU','SK_ID_CURR'], as_index=False).max()
    BUREAU_OVERDUE_NUM_36plus_0 = tmp[tmp.STATUS =='0'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_36plus_0'})
    BUREAU_OVERDUE_NUM_36plus_1 = tmp[tmp.STATUS == '1'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_36plus_1'})
    BUREAU_OVERDUE_NUM_36plus_2 = tmp[tmp.STATUS == '2'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_36plus_2'})
    BUREAU_OVERDUE_NUM_36plus_3 = tmp[tmp.STATUS == '3'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_36plus_3'})
    BUREAU_OVERDUE_NUM_36plus_4 = tmp[tmp.STATUS == '4'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_36plus_4'})
    BUREAU_OVERDUE_NUM_36plus_5 = tmp[tmp.STATUS == '5'][['SK_ID_BUREAU','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_BUREAU':'BUREAU_OVERDUE_NUM_36plus_5'})

    # 删除数据
    del bureau,bureau_balance,bureau_balance_union,tmp
    all_vars = locals()
    bureau_balance_extra = concat_df_by_name('BUREAU',all_vars)
    print(bureau_balance_extra.shape)
    return bureau_balance_extra

In [18]:
def process_pre():
    # previous_application表
    previous_application = load_data('previous_application')
    previous_application = reduce_mem_usage(previous_application)

    previous_application['RATE_INTEREST_ACTUAL'] = ((previous_application.AMT_ANNUITY * previous_application.CNT_PAYMENT) - previous_application.AMT_CREDIT) / previous_application.AMT_CREDIT
    previous_application.loc[previous_application.RATE_INTEREST_ACTUAL == -1.0,'RATE_INTEREST_ACTUAL'] = np.nan
    # 历史贷款总数
    PRE_CREDIT_NUM = previous_application[['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_NUM'})
    # 历史贷款总金额
    PRE_CREDIT_AMT = previous_application[['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_AMT'})
    # 历史贷款总月付款
    PRE_CREDIT_ANNUITY_AMT = previous_application[['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_ANNUITY_AMT'})
    # 历史刷卡贷款总数
    PRE_CREDIT_POS_NUM = previous_application[previous_application['NAME_PORTFOLIO'] == 'POS'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_POS_NUM'})
    # 历史刷卡贷款总金额
    PRE_CREDIT_POS_AMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'POS'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_POS_AMT'})
    # 历史刷卡贷款总月付款
    PRE_CREDIT_POS_ANNUITYAMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'POS'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_POS_ANNUITYAMT'})
    # 历史现金贷款总数
    PRE_CREDIT_CASH_NUM = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cash'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_CASH_NUM'})
    # 历史现金贷款总金额
    PRE_CREDIT_CASH_AMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cash'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_CASH_AMT'})
    # 历史现金贷款总月付款
    PRE_CREDIT_CASH_ANNUITYAMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cash'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_CASH_ANNUITYAMT'})
    # 历史其他贷款总数
    PRE_CREDIT_XNA_NUM = previous_application[previous_application['NAME_PORTFOLIO'] == 'XNA'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_XNA_NUM'})
    # 历史其他贷款总金额
    PRE_CREDIT_XNA_AMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'XNA'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_XNA_AMT'})
    # 历史其他贷款总月付款
    PRE_CREDIT_XNA_ANNUITYAMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'XNA'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_XNA_ANNUITYAMT'})
    # 历史信用卡贷款总数
    PRE_CREDIT_Cards_NUM = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cards'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_Cards_NUM'})
    # 历史信用卡贷款总金额
    PRE_CREDIT_Cards_AMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cards'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_Cards_AMT'})
    # 历史信用卡贷款总月付款
    PRE_CREDIT_Cards_ANNUITYAMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cards'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_Cards_ANNUITYAMT'})
    # 历史汽车贷款总数
    PRE_CREDIT_Cars_NUM = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cars'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_Cars_NUM'})
    # 历史汽车贷款总金额
    PRE_CREDIT_Cars_AMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cars'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_Cars_AMT'})
    # 历史汽车贷款总月付款
    PRE_CREDIT_Cars_ANNUITYAMT = previous_application[previous_application['NAME_PORTFOLIO'] == 'Cars'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_Cars_ANNUITYAMT'})
    # 历史通过贷款总数
    PRE_CREDIT_Approved_NUM = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Approved'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_Approved_NUM'})
    # 历史通过贷款总金额
    PRE_CREDIT_Approved_AMT = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Approved'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_Approved_AMT'})
    # 历史通过贷款总月付款
    PRE_CREDIT_Approved_ANNUITYAMT = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Approved'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_Approved_ANNUITYAMT'})
    # 历史取消贷款总数
    PRE_CREDIT_Canceled_NUM = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Canceled'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_Canceled_NUM'})
    # 历史取消贷款总金额
    PRE_CREDIT_Canceled_AMT = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Canceled'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_Canceled_AMT'})
    # 历史取消贷款总月付款
    PRE_CREDIT_Canceled_ANNUITYAMT = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Canceled'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_Canceled_ANNUITYAMT'})
    # 历史被拒绝贷款总数
    PRE_CREDIT_Refused_NUM = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Refused'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_Refused_NUM'})
    # 历史被拒绝贷款总金额
    PRE_CREDIT_Refused_AMT = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Refused'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_Refused_AMT'})
    # 历史被拒绝贷款总月付款
    PRE_CREDIT_Refused_ANNUITYAMT = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Refused'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_Refused_ANNUITYAMT'})
    # 历史未使用贷款总数
    PRE_CREDIT_Unused_NUM = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Unused offer'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_CREDIT_Unused_NUM'})
    # 历史未使用贷款总金额
    PRE_CREDIT_Unused_AMT = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Unused offer'][['SK_ID_CURR','AMT_CREDIT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_CREDIT':'PRE_CREDIT_Unused_AMT'})
    # 历史未使用贷款总月付款
    PRE_CREDIT_Unused_ANNUITYAMT = previous_application[previous_application['NAME_CONTRACT_STATUS'] == 'Unused offer'][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_CREDIT_Unused_ANNUITYAMT'})
    # 拒绝原因是HC的申请数
    PRE_HC_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'HC')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_HC_Refused_NUM'})
    # 拒绝原因是LIMIT的申请数
    PRE_LIMIT_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'LIMIT')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_LIMIT_Refused_NUM'})
    # 拒绝原因是SCO的申请数
    PRE_SCO_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'SCO')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_SCO_Refused_NUM'})
    # 拒绝原因是SCOFR的申请数
    PRE_SCOFR_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'SCOFR')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_SCOFR_Refused_NUM'})
    # 拒绝原因是XNA的申请数
    PRE_XNA_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'XNA')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_XNA_Refused_NUM'})
    # 拒绝原因是VERIF的申请数
    PRE_VERIF_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'VERIF')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_VERIF_Refused_NUM'})
    # 拒绝原因是SYSTEM的申请数
    PRE_SYSTEM_Refused_NUM = previous_application[(previous_application['NAME_CONTRACT_STATUS'] == 'Refused') & (previous_application['CODE_REJECT_REASON'] == 'SYSTEM')][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_SYSTEM_Refused_NUM'})
    # 历史申请最大利率
    PRE_MAX_INTEREST_RATE = previous_application[['SK_ID_CURR','RATE_INTEREST_ACTUAL']].groupby(['SK_ID_CURR']).max().rename(columns={'RATE_INTEREST_ACTUAL':'PRE_MAX_INTEREST_RATE'})
    # 历史申请最小利率
    PRE_MIN_INTEREST_RATE = previous_application[['SK_ID_CURR','RATE_INTEREST_ACTUAL']].groupby(['SK_ID_CURR']).min().rename(columns={'RATE_INTEREST_ACTUAL':'PRE_MIN_INTEREST_RATE'})
    # 历史申请平均利率
    PRE_AVG_INTEREST_RATE = previous_application[['SK_ID_CURR','RATE_INTEREST_ACTUAL']].groupby(['SK_ID_CURR']).mean().rename(columns={'RATE_INTEREST_ACTUAL':'PRE_AVG_INTEREST_RATE'})

    # 当前正在还的申请数
    PRE_REPAY_NUM = previous_application[previous_application.DAYS_TERMINATION == 365243.0][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_NUM_REPAY'})
    # 当前正在还的总金额
    PRE_REPAY_AMT = previous_application[previous_application.DAYS_TERMINATION == 365243.0][['SK_ID_CURR','AMT_APPLICATION']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_APPLICATION':'PRE_REPAY_AMT'})
    # 当前正在还的月付额
    PRE_REPAY_ANNUITYAMT = previous_application[previous_application.DAYS_TERMINATION == 365243.0][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_REPAY_ANNUITYAMT'})

    # 当前在还中属于逾期的申请数
    PRE_REAPY_OVERDUR_NUM = previous_application[(previous_application.DAYS_TERMINATION == 365243.0) & (previous_application.DAYS_LAST_DUE != 365243.0)][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'PRE_NUM_REAPY_OVERDUR'})
    # 当前正在还属于逾期的总金额
    PRE_REAPY_OVERDUR_AMT = previous_application[(previous_application.DAYS_TERMINATION == 365243.0) & (previous_application.DAYS_LAST_DUE != 365243.0)][['SK_ID_CURR','AMT_APPLICATION']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_APPLICATION':'PRE_REAPY_OVERDUR_AMT'})
    # 当前正在还属于逾期的月付额
    PRE_REAPY_OVERDUR_ANNUITYAMT = previous_application[(previous_application.DAYS_TERMINATION == 365243.0) & (previous_application.DAYS_LAST_DUE != 365243.0)][['SK_ID_CURR','AMT_ANNUITY']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_ANNUITY':'PRE_REAPY_OVERDUR_ANNUITYAMT'})

    # 最近3 6个月 1年 2年 3年 3年+贷款数 被拒绝数 通过数 贷款总金额 月付款额

    del previous_application
    all_vars = locals()
    pre_extra = concat_df_by_name('PRE',all_vars)
    print(pre_extra.shape)
    return pre_extra

In [19]:
def process_pos():
    # POS_CASH_balance表
    POS_CASH_balance = load_data('POS_CASH_balance')
    POS_CASH_balance = reduce_mem_usage(POS_CASH_balance)

    # pos总贷款数
    POS_CREDIT_NUM = POS_CASH_balance[['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'POS_CREDIT_NUM'})
    # pos已经还清的笔数
    POS_FINISH_NUM = POS_CASH_balance[POS_CASH_balance.NAME_CONTRACT_STATUS == 'Completed'][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'POS_FINISH_NUM'})
    # pos正在还的笔数
    POS_REPAY_NUM = POS_CASH_balance[ (POS_CASH_balance.MONTHS_BALANCE == -1) & (POS_CASH_balance.NAME_CONTRACT_STATUS == 'Active')][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'POS_REPAY_NUM'})
    # pos已经还清贷款最大逾期天数
    tmp = pd.DataFrame(POS_CASH_balance[POS_CASH_balance.NAME_CONTRACT_STATUS == 'Completed']['SK_ID_PREV'].unique(), columns=['SK_ID_PREV'])
    tmp = pd.merge(POS_CASH_balance, tmp, how='inner', on='SK_ID_PREV')
    POS_DAYS_MAXOVERDUE_FINISH = tmp[['SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_CURR']).max().rename(columns={'SK_DPD_DEF':'POS_DAYS_MAXOVERDUE_FINISH'})
    # pos已经还清贷款发生过逾期的笔数
    POS_NUM_MAXOVERDUE_FINISH = tmp[tmp.SK_DPD_DEF > 0][['SK_ID_PREV','SK_ID_CURR']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'POS_NUM_MAXOVERDUE_FINISH'})
    # pos正在还贷款最大逾期天数
    tmp = pd.DataFrame(POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE == -1) & (POS_CASH_balance.NAME_CONTRACT_STATUS == 'Active')]['SK_ID_PREV'].unique(), columns=['SK_ID_PREV'])
    tmp = pd.merge(POS_CASH_balance, tmp, how='inner', on='SK_ID_PREV')
    POS_DAYS_MAXOVERDUE_REPAY = tmp[['SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_CURR']).max().rename(columns={'SK_DPD_DEF':'POS_DAYS_MAXOVERDUE_REPAY'})
    # pos正在还贷款发生过逾期的笔数
    POS_NUM_MAXOVERDUE_REPAY = tmp[tmp.SK_DPD_DEF > 0][['SK_ID_PREV','SK_ID_CURR']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'POS_NUM_MAXOVERDUE_REPAY'})

    # pos最近6个月逾期0 7 14 30 90 90+天内的笔数
    tmp = POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE >= -6)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
    POS_OVERDUENUM_6_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_6_0'})
    POS_OVERDUENUM_6_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_6_7'})
    POS_OVERDUENUM_6_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_6_14'})
    POS_OVERDUENUM_6_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_6_30'})
    POS_OVERDUENUM_6_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_6_90'})
    POS_OVERDUENUM_6_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_6_90plus'})

    # pos最近7-12个月逾期0 7 14 30 90 90+天内的笔数
    tmp = POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE >= -12) & (POS_CASH_balance.MONTHS_BALANCE < -6)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
    POS_OVERDUENUM_12_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_12_0'})
    POS_OVERDUENUM_12_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_12_7'})
    POS_OVERDUENUM_12_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_12_14'})
    POS_OVERDUENUM_12_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_12_30'})
    POS_OVERDUENUM_12_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_12_90'})
    POS_OVERDUENUM_12_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_12_90plus'})

    # pos最近13-24个月逾期0 7 14 30 90 90+天内的笔数
    tmp = POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE >= -24) & (POS_CASH_balance.MONTHS_BALANCE < -12)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
    POS_OVERDUENUM_24_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_24_0'})
    POS_OVERDUENUM_24_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_24_7'})
    POS_OVERDUENUM_24_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_24_14'})
    POS_OVERDUENUM_24_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_24_30'})
    POS_OVERDUENUM_24_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_24_90'})
    POS_OVERDUENUM_24_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_24_90plus'})

    # pos最近24-36个月逾期0 7 14 30 90 90+天内的笔数
    tmp = POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE >= -36) & (POS_CASH_balance.MONTHS_BALANCE < -24)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
    POS_OVERDUENUM_36_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_36_0'})
    POS_OVERDUENUM_36_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_36_7'})
    POS_OVERDUENUM_36_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_36_14'})
    POS_OVERDUENUM_36_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_36_30'})
    POS_OVERDUENUM_36_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_36_90'})
    POS_OVERDUENUM_36_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_36_90plus'})

    # pos最近36以上个月逾期0 7 14 30 90 90+天内的笔数
    tmp = POS_CASH_balance[POS_CASH_balance.MONTHS_BALANCE < -36][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
    POS_OVERDUENUM_36plus_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_36plus_0'})
    POS_OVERDUENUM_36plus_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_36plus_7'})
    POS_OVERDUENUM_36plus_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_36plus_14'})
    POS_OVERDUENUM_36plus_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_36plus_30'})
    POS_OVERDUENUM_36plus_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_36plus_90'})
    POS_OVERDUENUM_36plus_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_OVERDUENUM_36plus_90plus'})

    # pos当前仍在逾期的笔数 最大逾期天数
    POS_NUM_OVERDUE_STILL = POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE == -1) & (POS_CASH_balance.SK_DPD_DEF > 30)][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'POS_NUM_OVERDUE_STILL'})
    POS_DAYS_MAXOVERDUE_STILL = POS_CASH_balance[(POS_CASH_balance.MONTHS_BALANCE == -1) & (POS_CASH_balance.SK_DPD_DEF > 30)][['SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_CURR']).max().rename(columns={'SK_DPD_DEF':'POS_DAYS_MAXOVERDUE_STILL'})

    del POS_CASH_balance,tmp
    all_vars = locals()
    pos_extra = concat_df_by_name('POS',all_vars)
    print(pos_extra.shape)
    return pos_extra

In [20]:
def process_inst():
    # installments_payments表
    installments_payments = load_data('installments_payments')
    installments_payments = reduce_mem_usage(installments_payments)

    # 分期还款记录中有很多应还款额为67.5的记录，中间有大量的逾期，明显有问题，我们剔除这些数据
    installments_payments = installments_payments[installments_payments.AMT_INSTALMENT > 100]
    installments_payments['DAYS_DIFF'] = installments_payments['DAYS_ENTRY_PAYMENT'] - installments_payments['DAYS_INSTALMENT']
    # 分期还款的贷款数
    INST_NUM = installments_payments[['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM'})
    # 分期还款最近6个月在还贷款数
    INST_NUM_6m = installments_payments[installments_payments.DAYS_INSTALMENT >= -180][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m'})
    # 分期还款最近6个月发生了逾期的贷款数
    INST_NUM_6m_all = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m_all'})
    # 分期还款最近6个月发生了逾期，逾期天数在7天内的贷款数
    INST_NUM_6m_7d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 0) & (installments_payments.DAYS_DIFF <= 7)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m_7d'})
    # 分期还款最近6个月发生了逾期，逾期天数在8-14天内的贷款数
    INST_NUM_6m_14d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 7) & (installments_payments.DAYS_DIFF <= 14)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m_14d'})
    # 分期还款最近6个月发生了逾期，逾期天数在15-30天内的贷款数
    INST_NUM_6m_30d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 14) & (installments_payments.DAYS_DIFF <= 30)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m_30d'})
    # 分期还款最近6个月发生了逾期，逾期天数在31-90天内的贷款数
    INST_NUM_6m_90d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 30) & (installments_payments.DAYS_DIFF <= 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m_90d'})
    # 分期还款最近6个月发生了逾期，逾期天数在90+天内的贷款数
    INST_NUM_6m_90dplus = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_6m_90dplus'})

    # 分期还款最近6-12个月在还贷款数
    INST_NUM_12m = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m'})
    # 分期还款最近6-12个月发生了逾期的贷款数
    INST_NUM_12m_all = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m_all'})
    # 分期还款最近6-12个月发生了逾期，逾期天数在7天内的贷款数
    INST_NUM_12m_7d = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 0) & (installments_payments.DAYS_DIFF <= 7)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m_7d'})
    # 分期还款最近6-12个月发生了逾期，逾期天数在8-14天内的贷款数
    INST_NUM_12m_14d = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 7) & (installments_payments.DAYS_DIFF <= 14)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m_14d'})
    # 分期还款最近6-12个月发生了逾期，逾期天数在15-30天内的贷款数
    INST_NUM_12m_30d = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 14) & (installments_payments.DAYS_DIFF <= 30)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m_30d'})
    # 分期还款最近6-12个月发生了逾期，逾期天数在31-90天内的贷款数
    INST_NUM_12m_90d = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 30) & (installments_payments.DAYS_DIFF <= 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m_90d'})
    # 分期还款最近6-12个月发生了逾期，逾期天数在90+天内的贷款数
    INST_NUM_12m_90dplus = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_12m_90dplus'})

    # 分期还款最近12-24个月在还贷款数
    INST_NUM_24m = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m'})
    # 分期还款最近12-24个月发生了逾期的贷款数
    INST_NUM_24m_all = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m_all'})
    # 分期还款最近12-24个月发生了逾期，逾期天数在7天内的贷款数
    INST_NUM_24m_7d = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_DIFF > 0) & (installments_payments.DAYS_DIFF <= 7)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m_7d'})
    # 分期还款最近12-24个月发生了逾期，逾期天数在8-14天内的贷款数
    INST_NUM_24m_14d = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_DIFF > 7) & (installments_payments.DAYS_DIFF <= 14)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m_14d'})
    # 分期还款最近12-24个月发生了逾期，逾期天数在15-30天内的贷款数
    INST_NUM_24m_30d = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_DIFF > 14) & (installments_payments.DAYS_DIFF <= 30)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m_30d'})
    # 分期还款最近12-24个月发生了逾期，逾期天数在31-90天内的贷款数
    INST_NUM_24m_90d = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_DIFF > 30) & (installments_payments.DAYS_DIFF <= 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m_90d'})
    # 分期还款最近12-24个月发生了逾期，逾期天数在90+天内的贷款数
    INST_NUM_24m_90dplus = installments_payments[(installments_payments.DAYS_INSTALMENT <= -720) & (installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_DIFF > 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_24m_90dplus'})

    # 分期还款最近24-36个月在还贷款数
    INST_NUM_36m = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m'})
    # 分期还款最近24-36个月发生了逾期的贷款数
    INST_NUM_36m_all = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m_all'})
    # 分期还款最近24-36个月发生了逾期，逾期天数在7天内的贷款数
    INST_NUM_36m_7d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_DIFF > 0) & (installments_payments.DAYS_DIFF <= 7)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m_7d'})
    # 分期还款最近24-36个月发生了逾期，逾期天数在8-14天内的贷款数
    INST_NUM_36m_14d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_DIFF > 7) & (installments_payments.DAYS_DIFF <= 14)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m_14d'})
    # 分期还款最近24-36个月发生了逾期，逾期天数在15-30天内的贷款数
    INST_NUM_36m_30d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_DIFF > 14) & (installments_payments.DAYS_DIFF <= 30)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m_30d'})
    # 分期还款最近24-36个月发生了逾期，逾期天数在31-90天内的贷款数
    INST_NUM_36m_90d = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_DIFF > 30) & (installments_payments.DAYS_DIFF <= 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m_90d'})
    # 分期还款最近24-36个月发生了逾期，逾期天数在90+天内的贷款数
    INST_NUM_36m_90dplus = installments_payments[(installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_DIFF > 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36m_90dplus'})

    # 分期还款最近36+个月在还贷款数
    INST_NUM_36mplus = installments_payments[installments_payments.DAYS_INSTALMENT < -1080][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus'})
    # 分期还款最近36+个月发生了逾期的贷款数
    INST_NUM_36mplus_all = installments_payments[installments_payments.DAYS_INSTALMENT < -1080 & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus_all'})
    # 分期还款最近36+个月发生了逾期，逾期天数在7天内的贷款数
    INST_NUM_36mplus_7d = installments_payments[installments_payments.DAYS_INSTALMENT < -1080 & (installments_payments.DAYS_DIFF > 0) & (installments_payments.DAYS_DIFF <= 7)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus_7d'})
    # 分期还款最近36+个月发生了逾期，逾期天数在8-14天内的贷款数
    INST_NUM_36mplus_14d = installments_payments[installments_payments.DAYS_INSTALMENT < -1080 & (installments_payments.DAYS_DIFF > 7) & (installments_payments.DAYS_DIFF <= 14)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus_14d'})
    # 分期还款最近36+个月发生了逾期，逾期天数在15-30天内的贷款数
    INST_NUM_36mplus_30d = installments_payments[installments_payments.DAYS_INSTALMENT < -1080 & (installments_payments.DAYS_DIFF > 14) & (installments_payments.DAYS_DIFF <= 30)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus_30d'})
    # 分期还款最近36+个月发生了逾期，逾期天数在31-90天内的贷款数
    INST_NUM_36mplus_90d = installments_payments[installments_payments.DAYS_INSTALMENT < -1080 & (installments_payments.DAYS_DIFF > 30) & (installments_payments.DAYS_DIFF <= 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus_90d'})
    # 分期还款最近36+个月发生了逾期，逾期天数在90+天内的贷款数
    INST_NUM_36mplus_90dplus = installments_payments[installments_payments.DAYS_INSTALMENT < -1080 & (installments_payments.DAYS_DIFF > 90)][['SK_ID_CURR','SK_ID_PREV']].groupby('SK_ID_CURR').nunique().rename(columns={'SK_ID_PREV':'INST_NUM_36mplus_90dplus'})

    # 分期还款最近6个月发生逾期的逾期金额
    INST_AMT_6m = installments_payments[(installments_payments.DAYS_INSTALMENT >= -180) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','AMT_PAYMENT']].groupby('SK_ID_CURR').sum().rename(columns={'AMT_PAYMENT':'INST_AMT_6m'})
    # 分期还款最近6-12个月发生逾期的逾期金额
    INST_AMT_12m = installments_payments[(installments_payments.DAYS_INSTALMENT < -180) & (installments_payments.DAYS_INSTALMENT >= -360) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','AMT_PAYMENT']].groupby('SK_ID_CURR').sum().rename(columns={'AMT_PAYMENT':'INST_AMT_12m'})
    # 分期还款最近12-24个月发生逾期的逾期金额
    INST_AMT_24m = installments_payments[(installments_payments.DAYS_INSTALMENT < -360) & (installments_payments.DAYS_INSTALMENT >= -720) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','AMT_PAYMENT']].groupby('SK_ID_CURR').sum().rename(columns={'AMT_PAYMENT':'INST_AMT_24m'})
    # 分期还款最近24-36个月发生逾期的逾期金额
    INST_AMT_36m = installments_payments[(installments_payments.DAYS_INSTALMENT < -720) & (installments_payments.DAYS_INSTALMENT >= -1080) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','AMT_PAYMENT']].groupby('SK_ID_CURR').sum().rename(columns={'AMT_PAYMENT':'INST_AMT_36m'})
    # 分期还款最近36+个月发生逾期的逾期金额
    INST_AMT_36mplus = installments_payments[(installments_payments.DAYS_INSTALMENT < -1080) & (installments_payments.DAYS_DIFF > 0)][['SK_ID_CURR','AMT_PAYMENT']].groupby('SK_ID_CURR').sum().rename(columns={'AMT_PAYMENT':'INST_AMT_36mplus'})

    # 分期还款当前仍在逾期的贷款数
    INST_NUM_STILL = installments_payments[installments_payments.DAYS_ENTRY_PAYMENT.isnull()][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'INST_NUM_STILL'})
    # 分期还款当前仍在逾期的贷款总逾期期数
    INST_NUM_SEQ_STILL = installments_payments[installments_payments.DAYS_ENTRY_PAYMENT.isnull()][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'INST_NUM_SEQ_STILL'})
    # 分期还款当前仍在逾期的贷款最大逾期天数
    INST_DAYS_MAX_STILL = installments_payments[installments_payments.DAYS_ENTRY_PAYMENT.isnull()][['SK_ID_CURR','DAYS_INSTALMENT']].groupby(['SK_ID_CURR']).min().rename(columns={'DAYS_INSTALMENT':'INST_DAYS_MAX_STILL'})
    # 分期还款当前仍在逾期的总逾期金额
    INST_AMT_STILL = installments_payments[installments_payments.DAYS_ENTRY_PAYMENT.isnull()][['SK_ID_CURR','AMT_INSTALMENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_INSTALMENT':'INST_AMT_STILL'})

    del installments_payments
    all_vars = locals()
    inst_extra = concat_df_by_name('INST',all_vars)
    print(inst_extra.shape)
    return inst_extra

In [21]:
def process_credit_balance():
    # credit_card_balance表
    credit_card_balance = load_data('credit_card_balance')
    credit_card_balance = reduce_mem_usage(credit_card_balance)

    # 信用卡数
    CREDIT_NUM = credit_card_balance[['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM'})
    # 不同状态的信用卡数
    CREDIT_NUM_ACTIVE = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Active'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM_ACTIVE'})
    CREDIT_NUM_COMPLETED = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Completed'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM_COMPLETED'})
    CREDIT_NUM_SIGNED = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Signed'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM_SIGNED'})
    CREDIT_NUM_DEMAND = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Demand'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM_DEMAND'})
    CREDIT_NUM_SENT = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Sent proposal'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM_SENT'})
    CREDIT_NUM_REFUSED = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Refused'][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).nunique().rename(columns={'SK_ID_PREV':'CREDIT_NUM_REFUSED'})
    # 信用卡使用时长
    tmp = credit_card_balance[['SK_ID_PREV','SK_ID_CURR','MONTHS_BALANCE']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).count()
    CREDIT_MONTHS_MAX = tmp[['SK_ID_CURR','MONTHS_BALANCE']].groupby('SK_ID_CURR').max().rename(columns={'MONTHS_BALANCE':'CREDIT_MONTHS_MAX'})
    CREDIT_MONTHS_MIN = tmp[['SK_ID_CURR','MONTHS_BALANCE']].groupby('SK_ID_CURR').min().rename(columns={'MONTHS_BALANCE':'CREDIT_MONTHS_MIN'})
    CREDIT_MONTHS_AVG = tmp[['SK_ID_CURR','MONTHS_BALANCE']].groupby('SK_ID_CURR').mean().rename(columns={'MONTHS_BALANCE':'CREDIT_MONTHS_AVG'})
    CREDIT_MONTHS_SUM = tmp[['SK_ID_CURR','MONTHS_BALANCE']].groupby('SK_ID_CURR').sum().rename(columns={'MONTHS_BALANCE':'CREDIT_MONTHS_SUM'})
    # 信用卡最近1 3 6 12 24 36 36+个月 月均余额
    CREDIT_AMT_1m = credit_card_balance[credit_card_balance.MONTHS_BALANCE == -1][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_1m'})
    CREDIT_AMT_3m = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -3][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_3m'})
    CREDIT_AMT_6m = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -6][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_6m'})
    CREDIT_AMT_12m = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -12][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_12m'})
    CREDIT_AMT_24m = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -24][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_24m'})
    CREDIT_AMT_36m = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -36][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_36m'})
    CREDIT_AMT_36mplus = credit_card_balance[credit_card_balance.MONTHS_BALANCE < -36][['SK_ID_CURR','AMT_BALANCE']].groupby(['SK_ID_CURR']).mean().rename(columns={'AMT_BALANCE':'CREDIT_AMT_36mplus'})
    # 信用卡最近1 3 6 12 24 36 36+个月取款金额
    CREDIT_AMT_1m_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE == -1][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_1m_CURRENT'})
    CREDIT_AMT_3m_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -3][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_3m_CURRENT'})
    CREDIT_AMT_6m_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -6][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_6m_CURRENT'})
    CREDIT_AMT_12m_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -12][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_12m_CURRENT'})
    CREDIT_AMT_24m_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -24][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_24m_CURRENT'})
    CREDIT_AMT_36m_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -36][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_36m_CURRENT'})
    CREDIT_AMT_36mplus_CURRENT = credit_card_balance[credit_card_balance.MONTHS_BALANCE < -36][['SK_ID_CURR','AMT_DRAWINGS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_CURRENT':'CREDIT_AMT_36mplus_CURRENT'})
    # 信用卡最近1 3 6 12 24 36 36+个月POS金额
    CREDIT_AMT_1m_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE == -1][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_1m_POS'})
    CREDIT_AMT_3m_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -3][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_3m_POS'})
    CREDIT_AMT_6m_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -6][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_6m_POS'})
    CREDIT_AMT_12m_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -12][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_12m_POS'})
    CREDIT_AMT_24m_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -24][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_24m_POS'})
    CREDIT_AMT_36m_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -36][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_36m_POS'})
    CREDIT_AMT_36mplus_POS = credit_card_balance[credit_card_balance.MONTHS_BALANCE < -36][['SK_ID_CURR','AMT_DRAWINGS_POS_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_POS_CURRENT':'CREDIT_AMT_36mplus_POS'})
    # 信用卡最近1 3 6 12 24 36 36+个月ATM金额
    CREDIT_AMT_1m_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE == -1][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_1m_ATM'})
    CREDIT_AMT_3m_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -3][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_3m_ATM'})
    CREDIT_AMT_6m_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -6][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_6m_ATM'})
    CREDIT_AMT_12m_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -12][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_12m_ATM'})
    CREDIT_AMT_24m_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -24][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_24m_ATM'})
    CREDIT_AMT_36m_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE >= -36][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_36m_ATM'})
    CREDIT_AMT_36mplus_ATM = credit_card_balance[credit_card_balance.MONTHS_BALANCE < -36][['SK_ID_CURR','AMT_DRAWINGS_ATM_CURRENT']].groupby(['SK_ID_CURR']).sum().rename(columns={'AMT_DRAWINGS_ATM_CURRENT':'CREDIT_AMT_36mplus_ATM'})
    # 信用卡最大 最小 平均额度
    CREDIT_LIMITAMT_MAX = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Active'][['SK_ID_CURR','AMT_CREDIT_LIMIT_ACTUAL']].groupby(['SK_ID_CURR']).max().rename(columns={'AMT_CREDIT_LIMIT_ACTUAL':'CREDIT_LIMITAMT_MAX'})
    CREDIT_LIMITAMT_MIN = credit_card_balance[credit_card_balance.NAME_CONTRACT_STATUS == 'Active'][['SK_ID_CURR','AMT_CREDIT_LIMIT_ACTUAL']].groupby(['SK_ID_CURR']).min().rename(columns={'AMT_CREDIT_LIMIT_ACTUAL':'CREDIT_LIMITAMT_MIN'})
    # 信用卡最近6个月逾期0 7 14 30 90 90+天内的笔数
    tmp = credit_card_balance[(credit_card_balance.MONTHS_BALANCE >= -6)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
    CREDIT_OVERDUENUM_6_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_6_0'})
    CREDIT_OVERDUENUM_6_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_6_7'})
    CREDIT_OVERDUENUM_6_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_6_14'})
    CREDIT_OVERDUENUM_6_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_6_30'})
    CREDIT_OVERDUENUM_6_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_6_90'})
    CREDIT_OVERDUENUM_6_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_6_90plus'})

    # 信用卡最近7-12个月逾期0 7 14 30 90 90+天内的笔数
    tmp = credit_card_balance[(credit_card_balance.MONTHS_BALANCE >= -12) & (credit_card_balance.MONTHS_BALANCE < -6)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
    CREDIT_OVERDUENUM_12_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_12_0'})
    CREDIT_OVERDUENUM_12_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_12_7'})
    CREDIT_OVERDUENUM_12_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_12_14'})
    CREDIT_OVERDUENUM_12_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_12_30'})
    CREDIT_OVERDUENUM_12_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_12_90'})
    CREDIT_OVERDUENUM_12_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_12_90plus'})

    # 信用卡最近13-24个月逾期0 7 14 30 90 90+天内的笔数
    tmp = credit_card_balance[(credit_card_balance.MONTHS_BALANCE >= -24) & (credit_card_balance.MONTHS_BALANCE < -12)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
    CREDIT_OVERDUENUM_24_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_24_0'})
    CREDIT_OVERDUENUM_24_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_24_7'})
    CREDIT_OVERDUENUM_24_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_24_14'})
    CREDIT_OVERDUENUM_24_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_24_30'})
    CREDIT_OVERDUENUM_24_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_24_90'})
    CREDIT_OVERDUENUM_24_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_24_90plus'})

    # 信用卡最近24-36个月逾期0 7 14 30 90 90+天内的笔数
    tmp = credit_card_balance[(credit_card_balance.MONTHS_BALANCE >= -36) & (credit_card_balance.MONTHS_BALANCE < -24)][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
    CREDIT_OVERDUENUM_36_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_36_0'})
    CREDIT_OVERDUENUM_36_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_36_7'})
    CREDIT_OVERDUENUM_36_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_36_14'})
    CREDIT_OVERDUENUM_36_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_36_30'})
    CREDIT_OVERDUENUM_36_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_36_90'})
    CREDIT_OVERDUENUM_36_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_36_90plus'})

    # 信用卡最近36以上个月逾期0 7 14 30 90 90+天内的笔数
    tmp = credit_card_balance[credit_card_balance.MONTHS_BALANCE < -36][['SK_ID_PREV','SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_PREV','SK_ID_CURR'], as_index=False).max()
    CREDIT_OVERDUENUM_36plus_0 = tmp[tmp.SK_DPD_DEF ==0][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_36plus_0'})
    CREDIT_OVERDUENUM_36plus_7 = tmp[(tmp.SK_DPD_DEF > 0) & (tmp.SK_DPD_DEF <= 7)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_36plus_7'})
    CREDIT_OVERDUENUM_36plus_14 = tmp[(tmp.SK_DPD_DEF > 7) & (tmp.SK_DPD_DEF <= 14)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_36plus_14'})
    CREDIT_OVERDUENUM_36plus_30 = tmp[(tmp.SK_DPD_DEF > 14) & (tmp.SK_DPD_DEF <= 30)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_36plus_30'})
    CREDIT_OVERDUENUM_36plus_90 = tmp[(tmp.SK_DPD_DEF > 30) & (tmp.SK_DPD_DEF <= 90)][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_36plus_90'})
    CREDIT_OVERDUENUM_36plus_90plus = tmp[tmp.SK_DPD_DEF > 90][['SK_ID_PREV','SK_ID_CURR']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':' CREDIT_OVERDUENUM_36plus_90plus'})

    # 信用卡当前仍在逾期的笔数 最大逾期天数
    # CREDIT_NUM_OVERDUE_STILL = credit_card_balance[(credit_card_balance.MONTHS_BALANCE == -1) & (credit_card_balance.SK_DPD_DEF > 30)][['SK_ID_CURR','SK_ID_PREV']].groupby(['SK_ID_CURR']).count().rename(columns={'SK_ID_PREV':'CREDIT_NUM_OVERDUE_STILL'})
    # CREDIT_DAYS_MAXOVERDUE_STILL = credit_card_balance[(credit_card_balance.MONTHS_BALANCE == -1) & (credit_card_balance.SK_DPD_DEF > 30)][['SK_ID_CURR','SK_DPD_DEF']].groupby(['SK_ID_CURR']).max().rename(columns={'SK_DPD_DEF':'CREDIT_DAYS_MAXOVERDUE_STILL'})

    del credit_card_balance,tmp
    all_vars = locals()
    credit_extra = concat_df_by_name('CREDIT',all_vars)
    print(credit_extra.shape)
    return credit_extra

In [22]:
bureau_extra = process_bureau()
bureau_balance_extra = process_bureau_balance()
pre_extra = process_pre()
pos_extra = process_pos()
inst_extra = process_inst()
credit_extra = process_credit_balance()

初始内存使用: 222.62 MB
优化后的内存使用: 126.04 MB
(305810, 43)
初始内存使用: 624.85 MB
优化后的内存使用: 442.60 MB
初始内存使用: 222.62 MB
优化后的内存使用: 126.04 MB
(130774, 42)
初始内存使用: 471.48 MB
优化后的内存使用: 321.75 MB
(338857, 46)
初始内存使用: 610.43 MB
优化后的内存使用: 314.76 MB
(337252, 39)
初始内存使用: 830.41 MB
优化后的内存使用: 415.20 MB
(339572, 45)
初始内存使用: 673.88 MB
优化后的内存使用: 318.63 MB
(103558, 71)


In [23]:
extra_info = [ bureau_extra, bureau_balance_extra, pre_extra, pos_extra, inst_extra, credit_extra ]

In [24]:
def merge_info(df,ls):
    print('合并前shape:{}'.format(df.shape))
    res = df.set_index('SK_ID_CURR')
    for extra in ls:
        res = pd.merge(res, extra, how = 'left', left_index=True, right_index=True)
        del extra
    print('合并后shape:{}'.format(res.shape))
    return res

In [25]:
app = train_set.copy().drop('TARGET', axis=1)
app_labels = train_set['TARGET'].copy()

In [26]:
app_extra = merge_info(app, extra_info)
del bureau_extra, bureau_balance_extra, pre_extra, pos_extra, inst_extra, credit_extra
del app,extra_info

合并前shape:(246008, 122)
合并后shape:(246008, 407)


### train again

In [44]:
# 筛选特征
cate_cols_extra = []
num_cols_extra = []
# 将类别属性放进去
cate_cols_extra.extend(app_extra.dtypes[app_extra.dtypes == 'object'].index.tolist())
# 将唯一值少于5个的数值型变量也放进去
cate_cols_extra.extend(select_low_cardinality_numeric_features(app_extra, 'TARGET'))
num_cols_extra.extend(app_extra.columns.difference(cate_cols_extra))
cat_cols_extra_object = app_extra[cate_cols_extra].select_dtypes(include=['object']).columns
cat_cols_extra_number = app_extra[cate_cols_extra].select_dtypes(include=['number']).columns

In [45]:
num_pipeline2 = make_pipeline(
                SimpleImputer(strategy='median'),
                StandardScaler()
                )
cat_number_pipeline2 = make_pipeline(
                FunctionTransformer(lambda X: X.astype(str),feature_names_out='one-to-one'),
                SimpleImputer(strategy='constant', fill_value='UKN'),
                OneHotEncoder(handle_unknown='ignore')
)
cat_object_pipeline2 = make_pipeline(
                SimpleImputer(strategy='constant', fill_value='UKN'),
                OneHotEncoder(handle_unknown='ignore')
)
processing2 = ColumnTransformer([
    ('num', num_pipeline2, num_cols_extra),
    ('cate_object',cat_object_pipeline2,cat_cols_extra_object),
    ('cate_number', cat_number_pipeline2, cat_cols_extra_number)
], remainder='passthrough')

In [46]:
app_transform2 = processing2.fit_transform(app_extra)
app_transform2.shape
app_transformed2 = pd.DataFrame(app_transform2, columns=processing2.get_feature_names_out())

(246008, 846)

In [47]:
# X_train, X_test, y_train, y_test = train_test_split(app_transformed2, app_labels,
#                                                     train_size=0.01, test_size=0.99)

# from tpot import TPOTRegressor
# # 初始化TPOTRegressor
# tpot = TPOTRegressor(
#     generations=50,
#     population_size=50,
#     offspring_size=50,
#     verbosity=2,
#     scoring='roc_auc',
#     max_time_mins=30,
#     n_jobs=-1,
#     random_state=42,
#     config_dict='TPOT light'
# )
# # 使用TPOTRegressor进行自动化机器学习
# tpot.fit(X_train, y_train)
# print(tpot.score(X_test, y_test))

In [48]:
lr2 = LinearRegression()
sgd2 = SGDRegressor()
tree2 = DecisionTreeRegressor()
gnb2 = GaussianNB()
nn2 = MLPRegressor()
models2 = [lr2, sgd2, tree2, gnb2, nn2]

In [49]:
# cross_validate_with_feature_importance(models2, app_transformed2, app_labels)
# del app,app_labels,app_transform2,app_transformed2

我们在上面加入了很多经验特征，并继续用粗糙的模型跑一遍，LR：0.7658 ± 0.0023 NN：0.7588 ± 0.0053，接下来我将对特征做一些变换，并加入交互特征

## feature transform and add ploy feature

In [50]:
from itertools import combinations

def generate_ratio_features(train_features, label, n):
    """
    将特征数据集任意两列相除，生成交互特征，并选择最好的n个特征
    :param train_features: 训练集特征数据集（DataFrame）
    :param label: 训练集标签列（Series）
    :param n: 要保留的新交互特征数
    :return: 选择最好的n个新交互特征
    """
    # 生成所有特征的组合
    features_combinations = list(combinations(train_features.columns, 2))
    features_combinations = [(a, b) if a < b else (b, a) for a, b in features_combinations]
    features_combinations = list(set(features_combinations))
    
    # 逐个生成交互特征并选择最好的n个特征
    top_ratio_features = pd.DataFrame()
    for i in range(0, len(features_combinations), 100):
        start_idx = i
        end_idx = min(i + 100, len(features_combinations))
        ratio_features = pd.DataFrame()
        for feature_pair in features_combinations[start_idx:end_idx]:
            numerator, denominator = feature_pair
            ratio_feature_name = '{}_div_{}'.format(numerator.replace("_", ""), denominator.replace("_", ""))
            if ratio_feature_name in ratio_features.columns:
                continue
            ratio_features[ratio_feature_name] = train_features[numerator] / train_features[denominator]
        ratio_features_merged = pd.concat([top_ratio_features, ratio_features], axis=1)
        top_features = evaluate_features(ratio_features_merged, label, n)
        top_ratio_features = ratio_features_merged[top_features]
    return top_ratio_features.columns

def evaluate_features(df, label, n):
    """
    对新生成的特征进行评估，挑选最好的n个
    """
    # Replace missing values with the median value of each column
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    for col in df.columns:
        median_value = df[col].median()
        df[col].fillna(median_value,inplace=True)

    # Scale the features
    scaler = StandardScaler()
    scaled_df = scaler.fit_transform(df)

    # Replace missing values with 0 after scaling
    scaled_df = np.nan_to_num(scaled_df)

    # Fit the decision tree regressor
    clf = DecisionTreeRegressor(random_state=0)
    clf.fit(scaled_df, label)

    feature_importance = pd.Series(clf.feature_importances_, index=df.columns)
    top_features = feature_importance.sort_values(ascending=False)[:n]
    return top_features.index.values

**时间相关的变量**

In [51]:
days_cols = []
for col in app_extra.columns:
    if 'DAYS' in col:
        days_cols.append(col)
# app_extra[days_cols].describe()

In [52]:
# days_inter_list = generate_ratio_features(app_extra[days_cols],app_labels,5).tolist()

the output looks like 'DAYSBIRTH_div_DAYSEMPLOYED'

**金额相关的变量**

In [53]:
amt_cols = []
for col in app_extra.columns:
    if 'AMT' in col and 'AMT_REQ' not in col:
        amt_cols.append(col)
# app_extra[amt_cols].describe()

In [54]:
# amt_inter_list = generate_ratio_features(app_extra[amt_cols],app_labels,15).tolist()

**次数相关变量**

In [55]:
num_cols = []
for col in app_extra.columns:
    if 'NUM' in col:
        num_cols.append(col)
# app_extra[num_cols].describe()

In [56]:
# num_inter_list = generate_ratio_features(app_extra[num_cols],app_labels,15).tolist()

**其他变量**

In [57]:
other_clos = []
tmp = days_cols + amt_cols + num_cols
for col in app.columns:
    if col not in tmp:
        other_clos.append(col)
other_clos.remove('SK_ID_CURR')
# app_extra[other_clos].describe()

**合并数据**

In [27]:
selected_inter_feature = ['DAYSBIRTH_div_DAYSEMPLOYED', 'DAYSBIRTH_div_DAYSIDPUBLISH',
'DAYSEMPLOYED_div_DAYSREGISTRATION', 'DAYSBIRTH_div_DAYSREGISTRATION','DAYSEMPLOYED_div_DAYSIDPUBLISH',
'AMTANNUITY_div_AMTCREDIT', 'AMTANNUITY_div_AMTGOODSPRICE',
'PRECREDITANNUITYAMT_div_PRECREDITApprovedAMT',
'AMTANNUITY_div_PRECREDITApprovedANNUITYAMT',
'PRECREDITAMT_div_PRECREDITANNUITYAMT',
'BUREAUDEBTAMT_div_BUREAULOANAMT', 'AMTCREDIT_div_PRECREDITApprovedAMT',
'PRECREDITPOSAMT_div_PRECREDITPOSANNUITYAMT',
'AMTINCOMETOTAL_div_PRECREDITPOSANNUITYAMT',
'AMTANNUITY_div_BUREAUAMTCONSUMER', 'AMTANNUITY_div_AMTINCOMETOTAL',
'AMTINCOMETOTAL_div_PRECREDITPOSAMT',
'AMTINCOMETOTAL_div_BUREAULOANAMT',
'AMTINCOMETOTAL_div_PRECREDITApprovedAMT',
'AMTCREDIT_div_AMTINCOMETOTAL',
'BUREAUNUMCONSUMER_div_PRECREDITNUM', 'BUREAUNUM_div_POSFINISHNUM',
'BUREAUNUM_div_PRECREDITNUM', 'BUREAUNUM_div_PRECREDITPOSNUM',
'POSOVERDUENUM120_div_PRECREDITNUM',
'INSTNUM36mplus7d_div_PRECREDITCASHNUM',
'BUREAUACTIVENUM_div_BUREAUNUMCARD',
'POSOVERDUENUM240_div_PRECREDITNUM', 'BUREAUNUMNORMAL_div_PRECREDITNUM',
'BUREAUNUM_div_BUREAUNUMDELAY', 'INSTNUM36m_div_INSTNUM36mplus90dplus',
'PRECREDITApprovedNUM_div_PRECREDITPOSNUM',
'PRECREDITNUM_div_PRECREDITXNANUM',
'BUREAUOVERDUENUM120_div_PRECREDITNUM', 'BUREAUClosedNUM_div_INSTNUM'
                         ]
# selected_inter_feature = days_inter_list + amt_inter_list + num_inter_list
def gen_inter_feature(df, features):
    dic = {}
    for col in df.columns:
        dic[col] = col.replace('_','')
    df.rename(columns = dic, inplace=True)
    
    result = pd.DataFrame()
    for feature in features:
        numerator, denominator = feature.split('_div_')
        result[feature] = df[numerator] / df[denominator]
        result.replace([np.inf, -np.inf], 999, inplace=True) # 分母有0的情况
    res = pd.concat([df, result], axis=1)
    return res

In [28]:
app_extra_add = gen_inter_feature(app_extra, selected_inter_feature)

In [29]:
app_extra_add.shape

(246008, 442)

### train again

In [30]:
# 筛选特征
cate_cols = []
num_cols = []
# 将类别属性放进去
cate_cols.extend(app_extra_add.dtypes[app_extra_add.dtypes == 'object'].index.tolist())
# 将唯一值少于5个的数值型变量也放进去
cate_cols.extend(select_low_cardinality_numeric_features(app_extra_add, 'TARGET'))
num_cols.extend(app_extra_add.columns.difference(cate_cols))
cat_cols_object = app_extra_add[cate_cols].select_dtypes(include=['object']).columns
cat_cols_number = app_extra_add[cate_cols].select_dtypes(include=['number']).columns

num_pipeline3 = make_pipeline(
                SimpleImputer(strategy='median'),
                StandardScaler()
                )
cat_number_pipeline3 = make_pipeline(
                FunctionTransformer(lambda X: X.astype(str),feature_names_out='one-to-one'),
                SimpleImputer(strategy='constant', fill_value='UKN'),
                OneHotEncoder(handle_unknown='ignore')
)
cat_object_pipeline3 = make_pipeline(
                SimpleImputer(strategy='constant', fill_value='UKN'),
                OneHotEncoder(handle_unknown='ignore')
)
processing3 = ColumnTransformer([
    ('num', num_pipeline3, num_cols),
    ('cate_object',cat_object_pipeline3,cat_cols_object),
    ('cate_number', cat_number_pipeline3, cat_cols_number)
], remainder='passthrough')

app_transform3 = processing3.fit_transform(app_extra_add)
app_transform3.shape
app_transformed3 = pd.DataFrame(app_transform3, columns=processing3.get_feature_names_out())

(246008, 881)

In [62]:
lr3 = LinearRegression()
sgd3 = SGDRegressor()
tree3 = DecisionTreeRegressor()
gnb3 = GaussianNB()
nn3 = MLPRegressor()
models3 = [lr3, sgd3, tree3, gnb3, nn3]

# cross_validate_with_feature_importance(models3, app_transformed3, app_labels)

LR:0.7676 ± 0.0027 NN:0.7540 ± 0.0050,加入交互特征后，LR表现略微好点，NN略微下降，可能是因为默认参数使得模型太过粗糙导致

## delete invaild feature

In [63]:
# from sklearn.feature_selection import SequentialFeatureSelector
# from sklearn.feature_selection import RFECV

# lr_tmp = LinearRegression()
# sfs = SequentialFeatureSelector(lr_tmp,n_jobs=-1)
# lr_tmp = LinearRegression()
# selector = RFECV(lr_tmp, scoring='roc_auc',  step=1, min_features_to_select=400, cv=5)
# selector = selector.fit(app_transformed3, app_labels)
# app_transformed4 = sfs.transform(app_transformed3)
# # 这段太耗费计算资源

In [33]:
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.feature_selection import SelectFromModel

erg = ExtraTreesRegressor(n_jobs=-1)
erg = erg.fit(app_transformed3, app_labels)
model = SelectFromModel(erg, prefit=True)
app_transformed4 = model.transform(app_transformed3)

In [52]:
app_transformed4 = pd.DataFrame(app_transformed4, columns=model.get_feature_names_out(input_features=app_transformed3.columns))

In [55]:
app_transformed4.shape

(246008, 314)

In [54]:
lr4 = LinearRegression()
sgd4 = SGDRegressor()
tree4 = DecisionTreeRegressor()
gnb4 = GaussianNB()
nn4 = MLPRegressor()
models4 = [lr4, sgd4, tree4, gnb4, nn4]

cross_validate_with_feature_importance(models4, app_transformed4, app_labels)

Model 1: LinearRegression
Fold 1
Training time: 2.23s
Validation ROC AUC score: 0.7734
Fold 2
Training time: 1.98s
Validation ROC AUC score: 0.7687
Fold 3
Training time: 1.97s
Validation ROC AUC score: 0.7678
Fold 4
Training time: 1.97s
Validation ROC AUC score: 0.7637
Fold 5
Training time: 1.96s
Validation ROC AUC score: 0.7629
Mean ROC AUC score: 0.7673 ± 0.0038
Model 2: SGDRegressor
Fold 1
Training time: 1.03s
Validation ROC AUC score: 0.5065
Fold 2
Training time: 1.03s
Validation ROC AUC score: 0.5073
Fold 3
Training time: 1.04s
Validation ROC AUC score: 0.4906
Fold 4
Training time: 1.02s
Validation ROC AUC score: 0.5284
Fold 5
Training time: 1.02s
Validation ROC AUC score: 0.4896
Mean ROC AUC score: 0.5045 ± 0.0141
Model 3: DecisionTreeRegressor
Fold 1
Training time: 44.69s
Validation ROC AUC score: 0.5461
Fold 2
Training time: 46.19s
Validation ROC AUC score: 0.5447
Fold 3
Training time: 44.12s
Validation ROC AUC score: 0.5418
Fold 4
Training time: 44.40s
Validation ROC AUC score

In [57]:
del app_extra, app_extra_add, app_transform3
gc.collect()

4068

## try more model and fine tune

In [59]:
from lightgbm import LGBMClassifier

clf = LGBMClassifier(
            nthread=4,
            n_estimators=10000,
            learning_rate=0.02,
            num_leaves=34,
            colsample_bytree=0.9497036,
            subsample=0.8715623,
            max_depth=8,
            reg_alpha=0.041545473,
            reg_lambda=0.0735294,
            min_split_gain=0.0222415,
            min_child_weight=39.3259775,
            silent=-1,
            verbose=-1, )

In [73]:
import re
app_transformed3 = app_transformed3.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [71]:
cross_validate_with_feature_importance([clf], app_transformed4, app_labels)

Model 1: LGBMClassifier
Fold 1
Training time: 194.60s
Validation ROC AUC score: 0.7746
Fold 2
[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
Training time: 192.97s
Validation ROC AUC score: 0.7690
Fold 3
[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
Training time: 192.85s
Validation ROC AUC score: 0.7655
Fold 4
[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
Training time: 198.75s
Validation ROC AUC score: 0.7665
Fold 5
[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
Training time: 196.69s
Validation ROC AUC score: 0.7668
Top 10 most important features:
num__EXTSOURCE2                       31318
num__DAYSBIRTH                        27589
num__AMTANNUITY_div_AMTCREDIT         26705
num__DAYSLASTPHONECHA

In [66]:
app_transformed4.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in app_transformed4.columns]

In [74]:
cross_validate_with_feature_importance([clf], app_transformed3, app_labels)

Model 1: LGBMClassifier
Fold 1
Training time: 222.26s
Validation ROC AUC score: 0.7739
Fold 2
[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
Training time: 227.28s
Validation ROC AUC score: 0.7708
Fold 3
[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
Training time: 220.89s
Validation ROC AUC score: 0.7662
Fold 4
[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
Training time: 273.28s
Validation ROC AUC score: 0.7680
Fold 5
[LightGBM] [Warning] num_threads is set with nthread=4, will be overridden by n_jobs=-1. Current value: num_threads=-1
Training time: 222.52s
Validation ROC AUC score: 0.7687
Top 10 most important features:
num__EXTSOURCE2                       29251
num__DAYSBIRTH                        26304
num__AMTANNUITY_div_AMTCREDIT         25122
num__DAYSLASTPHONECHA

线性模型

In [61]:
from sklearn.linear_model import Ridge
clf = Ridge(alpha=750)
cross_validate_with_feature_importance([clf], app_transformed3, app_labels)

Model 1: Ridge
Fold 1
Training time: 2.21s
Validation ROC AUC score: 0.7755
Fold 2
Training time: 2.21s
Validation ROC AUC score: 0.7704
Fold 3
Training time: 2.21s
Validation ROC AUC score: 0.7693
Fold 4
Training time: 2.22s
Validation ROC AUC score: 0.7674
Fold 5
Training time: 2.32s
Validation ROC AUC score: 0.7662
Mean ROC AUC score: 0.7697 ± 0.0032


In [68]:
from sklearn.linear_model import Lasso
clf = Lasso(alpha=1e-5)
cross_validate_with_feature_importance([clf], app_transformed3, app_labels)

Model 1: Lasso
Fold 1
Training time: 81.15s
Validation ROC AUC score: 0.7728
Fold 2
Training time: 82.25s
Validation ROC AUC score: 0.7691
Fold 3
Training time: 84.13s
Validation ROC AUC score: 0.7659
Fold 4
Training time: 82.83s
Validation ROC AUC score: 0.7665
Fold 5
Training time: 83.93s
Validation ROC AUC score: 0.7639
Mean ROC AUC score: 0.7677 ± 0.0031


In [70]:
from sklearn.linear_model import ElasticNet
reg = ElasticNet(alpha=1e-5, l1_ratio=0.5)
cross_validate_with_feature_importance([reg], app_transformed3, app_labels)

Model 1: ElasticNet
Fold 1
Training time: 57.29s
Validation ROC AUC score: 0.7749
Fold 2
Training time: 54.67s
Validation ROC AUC score: 0.7698
Fold 3
Training time: 55.55s
Validation ROC AUC score: 0.7680
Fold 4
Training time: 51.77s
Validation ROC AUC score: 0.7670
Fold 5
Training time: 52.76s
Validation ROC AUC score: 0.7654
Mean ROC AUC score: 0.7690 ± 0.0032


In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'alpha':[1e-7,1e-6,1e-5], 'l1_ratio':[0.1, 0.5, 0.9]}

gs = GridSearchCV(reg, parameters, scoring='roc_auc', n_jobs=-1, cv=4 )

gs.fit(app_transformed3, app_labels)

In [ ]:
gs.best_estimator_
gs.best_score_
gs.best_params_

## Evaluate model on test set

## submit result